In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.vgg16 import VGG16

import numpy as np

In [25]:

# 기존 이미지  갯수 늘리기:전처리
# train_data_gen = ImageDataGenerator(rescale=1./255)
# train_generator = train_data_gen.flow_from_directory('../fig/my_images/train', target_size=(150, 150), batch_size=5, class_mode='binary')

train_data_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_data_gen.flow_from_directory('./data/train', target_size=(150, 150), batch_size=5, class_mode='binary')

test_data_gen = ImageDataGenerator(rescale=1./255)  # 테스트 데이터는 정규화만
test_generator = test_data_gen.flow_from_directory('./data/test', target_size=(150, 150), batch_size=5, class_mode='binary')

Found 218 images belonging to 2 classes.
Found 90 images belonging to 2 classes.


In [26]:
## transfer learning
transfer_vgg16 = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3))
transfer_vgg16.trainable = False
transfer_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [27]:
finetune_vgg16 = Sequential()
finetune_vgg16.add(transfer_vgg16)
finetune_vgg16.add(Flatten())
finetune_vgg16.add(Dense(64, activation='relu'))
finetune_vgg16.add(Dropout(0.3))
finetune_vgg16.add(Dense(1, activation='sigmoid'))
finetune_vgg16.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_6 (Dense)             (None, 64)                524352    
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 15,239,105
Trainable params: 524,417
Non-trainable params: 14,714,688
_________________________________________________________________


In [28]:
finetune_vgg16.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['acc'])

checkpoint = ModelCheckpoint(filepath = './data/reco_family.hdf5', save_best_only=True)
erarly_stop = EarlyStopping(patience=10)

In [32]:
history = finetune_vgg16.fit(train_generator, epochs=100, batch_size = 20, validation_data=test_generator, callbacks=[checkpoint, erarly_stop])

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

In [ ]:
finetune_vgg16.save('./reco_family_1.hdf5')